# Introduction to the RockBlock and LayeredRockBlock objects

The RockBlock and LayeredRockBlock are two landlab objects meant to make it easier to work with spatially variable lithology that produces spatially variable parameter values (e.g. stream power erodability and diffusivity). 

In this tutorial we will first use the LayeredRockBlock to erode either dipping layeres or an anticline. Then we will use RockBlock to create inverted topography. 

To start, we will import the necessary modules. A note: this tutorial uses the [HoloViews package](http://holoviews.org) for visualization. This package is a great tool for dealing with multidimentional annotated data (e.g. an xarray dataset). If you get an error on import, consider updating dask (this is what the author needed to do in April 2018).  

In [ ]:
import os
import numpy as np
import xarray as xr
import dask
%matplotlib inline

import holoviews as hv
hv.extension('bokeh', 'matplotlib')

from landlab import RasterModelGrid
from landlab.layers import RockBlock, LayeredRockBlock
from landlab.components import FlowAccumulator, FastscapeEroder
from landlab.plot import imshow_grid

## Part 1: Eroding layered rock

First we will create an instance of a LayedRockBlock. Both LayeredRockBlock and RockBlock work closely with a landlab ModelGrid and stores information about rock type at each node. 

To create a LayeredRock you need the following information:

1. A dictionary of rock property attributes. 
2. A model grid that has the field `'topographic__elevation'` already created.
3. A list of elevations that your layers will go through at specified anchor point (default value for the anchor point is (0, 0). 
4. A functional form in two variables (x and y) that defines the shape of your surface. 

The use of this function form makes it possible for any function of x and y to be passed to Layered Rock Block.
In this example the option for an anticline is currently uncommented, though options for steep and shallow dipping layers are also provided. 

In [ ]:
attrs = {'K_sp': {0: 0.0003,
                  1: 0.0001}}

mg = RasterModelGrid((50, 50), 100)
z = mg.add_zeros('node', 'topographic__elevation') 
random_field = 0.01*np.random.randn(mg.size('node'))
z += random_field - random_field.min()

z0s = 50 * np.arange(-20, 20)
z0s[-1] = z0s[-2] + 10000
ids = np.tile([0,1], 20) 

# Anticline
rb = LayeredRockBlock(mg, z0s, ids, x0=2500, y0=2500, function = lambda x, y : ((0.005*x)**2+(0.005*y)**2), attrs=attrs)

# Shallow dips
#rb = LayeredRockBlock(mg, z0s, ids, x0=5000, y0=5000, function = lambda x, y : ((0.001*x)+(0.003*y)), attrs=attrs)

# Steeper dips
#rb = LayeredRockBlock(mg, z0s, ids, x0=5000, y0=5000, function = lambda x, y : ((0.01*x)+(0.01*y)), attrs=attrs)

Now that we've created the LayeredRockBlock, model grid fields for each of the LayeredRockBlock attributes exist and have been set to the values of the rock exposed at the surface. 

Here we plot the value of `'K_sp'` as a function of the model grid. 

In [ ]:
imshow_grid(mg, 'K_sp')

As you can see (in the default anticline option) we have concentric elipses of stronger and weaker rock. 

Next, lets instantiate a FlowAccumulator and a FastscapeEroder to create a simple landscape evolution model. 

We will point the FastscapeEroder to the model grid field `'K_sp'` so that it will respond to the spatially variable erodabilities created by the LayeredRockBlock. 

We will also instatiate an xarray dataset used to store the output of our model through time for visualization. 

In [ ]:
nts = 400
U = 0.001
dt = 1000

fa = FlowAccumulator(mg)
sp = FastscapeEroder(mg, K_sp='K_sp')

out_fields = ['topographic__elevation',
              'K_sp', 
              'rock_type__id']

ds = xr.Dataset(data_vars={'topographic__elevation' : (('y', 'x', 't'), np.empty((mg.shape[0], mg.shape[1], nts))),
                           'K_sp' : (('y', 'x', 't'), np.empty((mg.shape[0], mg.shape[1], nts))),
                           'rock_type__id': (('y', 'x', 't'), np.empty((mg.shape[0], mg.shape[1], nts)))},
                coords={'x': mg.x_of_node.reshape(mg.shape)[0,:],
                        'y': mg.y_of_node.reshape(mg.shape)[:, 1],
                        't': dt*np.arange(nts)/1e6})

Here we run the model. In each time step we first run the FlowAccumulator to direct flow and accumulatate drainage area. Then the FastscapeEroder erodes the topography based on the stream power equation using the erodability value in the field`'K_sp'`. We create an uplift field that uplifts only the model grid's core nodes. After uplifting these core nodes, we update the LayeredRockBlock. Importantly, we must tell the LayeredRockBlock how it has been advected upward by uplift. 

Within each timestep we save information about the model for plotting. 

In [ ]:
for i in range(nts):
    fa.run_one_step()
    sp.run_one_step(dt = dt)
    dz_ad = np.zeros(mg.size('node'))
    dz_ad[mg.core_nodes] = U * dt
    z += dz_ad
    rb.run_one_step(dz_advection = dz_ad)
    
    for of in out_fields:
        ds[of][:,:,i] = mg['node'][of].reshape(mg.shape)


Now that the model has run, lets start by plotting the resulting topography. 

In [ ]:
imshow_grid(mg, 'topographic__elevation', cmap='viridis')

The layers of rock clearly influence the form of topography. 

Next we will use HoloViews to visualize the topography and rock type together. 

To start, we create a HoloViewDataset from our xarray datastructure. 

In [ ]:
hvds = hv.Dataset(ds)
hvds

Next we specify that we want two images, one showing rock type and one showing topographic elevation. A slider bar 

In [ ]:
%opts Image (cmap='viridis')

topo = hvds.to(hv.Image, ['x', 'y'], ['topographic__elevation'])
rock = hvds.to(hv.Image, ['x', 'y'], ['rock_type__id'])

topo + rock

Make inverted topography by filling RockBlock in with resistant material only where the DA is large. 

In [ ]:
mg2 = RasterModelGrid((50, 50), 100)
z2 = mg2.add_zeros('node', 'topographic__elevation') 
random_field = 0.01*np.random.randn(mg2.size('node'))
z2 += random_field - random_field.min()

thicknesses2 = [1000]
ids2 =[0]

attrs2 = {'K_sp': {0: 0.0001,
                   1: 0.00001}}

rb2 = RockBlock(mg2, thicknesses2, ids2, attrs=attrs2)

nts = 400
U = 0.001
dt = 1000

fa2 = FlowAccumulator(mg2)
sp2 = FastscapeEroder(mg2, K_sp='K_sp')

out_fields = ['topographic__elevation',
              'K_sp', 
              'rock_type__id']

ds2 = xr.Dataset(data_vars={'topographic__elevation' : (('y', 'x', 't'), np.empty((mg2.shape[0], mg2.shape[1], nts))),
                            'K_sp' : (('y', 'x', 't'), np.empty((mg2.shape[0], mg2.shape[1], nts))),
                            'rock_type__id': (('y', 'x', 't'), np.empty((mg2.shape[0], mg2.shape[1], nts)))},
                 coords={'x': mg2.x_of_node.reshape(mg2.shape)[0,:],
                         'y': mg2.y_of_node.reshape(mg2.shape)[:, 1],
                         't': dt*np.arange(nts)/1e6})
half_nts = int(nts/2)
for i in range(half_nts):
    fa2.run_one_step()
    sp2.run_one_step(dt = dt)
    dz_ad2 = np.zeros(mg2.size('node'))
    dz_ad2[mg2.core_nodes] = U * dt
    z2 += dz_ad2
    rb2.run_one_step(dz_advection = dz_ad2)
    
    for of in out_fields:
        ds2[of][:,:,i] = mg2['node'][of].reshape(mg2.shape)


In [ ]:
imshow_grid(mg2, 'topographic__elevation')

In [ ]:
volcano_deposits = np.zeros(mg2.size('node'))
da_big_enough = mg2['node']['drainage_area']>1e4
volcano_deposits[da_big_enough] = 30.0
volcano_deposits[mg2.boundary_nodes] = 0.0
rb2.add_layer(volcano_deposits, rock_id=1)

imshow_grid(mg2, da_big_enough)

In [ ]:
for i in range(half_nts, nts):
    fa2.run_one_step()
    sp2.run_one_step(dt = dt)
    dz_ad = np.zeros(mg2.size('node'))
    dz_ad[mg2.core_nodes] = U * dt
    z2 += dz_ad
    rb2.run_one_step(dz_advection = dz_ad)
    
    for of in out_fields:
        ds2[of][:,:,i] = mg2['node'][of].reshape(mg2.shape)

In [ ]:
imshow_grid(mg2, 'topographic__elevation')

In [ ]:
%opts Image (cmap='viridis')

hvds2 = hv.Dataset(ds2)
topo2 = hvds2.to(hv.Image, ['x', 'y'], ['topographic__elevation'])
rock2 = hvds2.to(hv.Image, ['x', 'y'], ['rock_type__id'])

topo2 + rock2